In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import medfilt, welch, find_peaks
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from scipy.stats import skew, kurtosis
import pywt
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the data
df = pd.read_csv('/content/DataLoader.csv')

# Map 'F' to 0 and 'M' to 1 in the 'Gender' column to make it numeric
df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})

# Preprocessing
# Convert string representations of arrays to actual lists of numbers
df['ICA_Output'] = df['ICA_Output'].apply(eval)
df['CHROME_Output'] = df['CHROME_Output'].apply(eval)
df['POS_Output'] = df['POS_Output'].apply(eval)

# Pad or truncate sequences to a common length
max_length = max(len(seq) for seq in df['ICA_Output'])
X_ica_padded = pad_sequences(df['ICA_Output'].to_list(), maxlen=max_length, padding='post', dtype='float32')
X_chrome_padded = pad_sequences(df['CHROME_Output'].to_list(), maxlen=max_length, padding='post', dtype='float32')
X_pos_padded = pad_sequences(df['POS_Output'].to_list(), maxlen=max_length, padding='post', dtype='float32')

# Apply filtering, normalization, baseline correction
filtered_ica_array = []
filtered_chrome_array = []
filtered_pos_array = []

for signal_ica, signal_chrome, signal_pos in zip(X_ica_padded, X_chrome_padded, X_pos_padded):
    # Apply median filtering
    filtered_ica_signal = medfilt(signal_ica, kernel_size=3)
    filtered_chrome_signal = medfilt(signal_chrome, kernel_size=3)
    filtered_pos_signal = medfilt(signal_pos, kernel_size=3)

    # Normalize the signals
    normalized_ica_signal = (filtered_ica_signal - np.mean(filtered_ica_signal)) / np.std(filtered_ica_signal)
    normalized_chrome_signal = (filtered_chrome_signal - np.mean(filtered_chrome_signal)) / np.std(filtered_chrome_signal)
    normalized_pos_signal = (filtered_pos_signal - np.mean(filtered_pos_signal)) / np.std(filtered_pos_signal)

    # Apply baseline correction if needed
    baseline_corrected_ica_signal = normalized_ica_signal - np.mean(normalized_ica_signal)
    baseline_corrected_chrome_signal = normalized_chrome_signal - np.mean(normalized_chrome_signal)
    baseline_corrected_pos_signal = normalized_pos_signal - np.mean(normalized_pos_signal)

    filtered_ica_array.append(baseline_corrected_ica_signal)
    filtered_chrome_array.append(baseline_corrected_chrome_signal)
    filtered_pos_array.append(baseline_corrected_pos_signal)

filtered_ica_array = np.array(filtered_ica_array)
filtered_chrome_array = np.array(filtered_chrome_array)
filtered_pos_array = np.array(filtered_pos_array)

# Feature Engineering: Add statistical features to the input data
statistical_features = np.hstack([
    np.mean(filtered_ica_array, axis=1).reshape(-1, 1),
    np.mean(filtered_chrome_array, axis=1).reshape(-1, 1),
    np.mean(filtered_pos_array, axis=1).reshape(-1, 1),
    np.median(filtered_ica_array, axis=1).reshape(-1, 1),
    np.median(filtered_chrome_array, axis=1).reshape(-1, 1),
    np.median(filtered_pos_array, axis=1).reshape(-1, 1),
    np.var(filtered_ica_array, axis=1).reshape(-1, 1),
    np.var(filtered_chrome_array, axis=1).reshape(-1, 1),
    np.var(filtered_pos_array, axis=1).reshape(-1, 1),
    skew(filtered_ica_array, axis=1).reshape(-1, 1),
    skew(filtered_chrome_array, axis=1).reshape(-1, 1),
    skew(filtered_pos_array, axis=1).reshape(-1, 1),
    kurtosis(filtered_ica_array, axis=1).reshape(-1, 1),
    kurtosis(filtered_chrome_array, axis=1).reshape(-1, 1),
    kurtosis(filtered_pos_array, axis=1).reshape(-1, 1)
])

# Compute frequency domain features
frequency_features_ica = []
frequency_features_chrome = []
frequency_features_pos = []

for signal_ica, signal_chrome, signal_pos in zip(filtered_ica_array, filtered_chrome_array, filtered_pos_array):
    _, psd_ica = welch(signal_ica, nperseg=256)
    _, psd_chrome = welch(signal_chrome, nperseg=256)
    _, psd_pos = welch(signal_pos, nperseg=256)

    frequency_features_ica.append([np.mean(psd_ica), np.median(psd_ica), np.max(psd_ica), np.min(psd_ica), skew(psd_ica), kurtosis(psd_ica)])
    frequency_features_chrome.append([np.mean(psd_chrome), np.median(psd_chrome), np.max(psd_chrome), np.min(psd_chrome), skew(psd_chrome), kurtosis(psd_chrome)])
    frequency_features_pos.append([np.mean(psd_pos), np.median(psd_pos), np.max(psd_pos), np.min(psd_pos), skew(psd_pos), kurtosis(psd_pos)])

frequency_features_ica = np.array(frequency_features_ica)
frequency_features_chrome = np.array(frequency_features_chrome)
frequency_features_pos = np.array(frequency_features_pos)

# Wavelet Transform features
def wavelet_features(signal, max_length=6):
    coeffs, _ = pywt.cwt(signal, np.arange(1, 128), 'morl')
    # Extract features from the coefficients
    mean_coeff = np.mean(coeffs, axis=1)
    median_coeff = np.median(coeffs, axis=1)
    max_coeff = np.max(coeffs, axis=1)
    min_coeff = np.min(coeffs, axis=1)
    skewness_coeff = skew(coeffs, axis=1)
    kurtosis_coeff = kurtosis(coeffs, axis=1)
    # Pad or truncate features to a fixed length
    padded_features = [mean_coeff, median_coeff, max_coeff, min_coeff, skewness_coeff, kurtosis_coeff]
    padded_features = [np.pad(f, (0, max_length - len(f)), mode='constant') if len(f) < max_length else f[:max_length] for f in padded_features]
    return np.array(padded_features)

wavelet_features_list_ica = [wavelet_features(seq) for seq in filtered_ica_array]
wavelet_features_list_chrome = [wavelet_features(seq) for seq in filtered_chrome_array]
wavelet_features_list_pos = [wavelet_features(seq) for seq in filtered_pos_array]

wavelet_features_array_ica = np.array(wavelet_features_list_ica)
wavelet_features_array_chrome = np.array(wavelet_features_list_chrome)
wavelet_features_array_pos = np.array(wavelet_features_list_pos)

# Combine all features
X_features = np.hstack([statistical_features, frequency_features_ica, frequency_features_chrome, frequency_features_pos])

# Add age and gender to features
X_with_age_gender = np.hstack([X_features, df[['Age', 'Gender']].values])

# Split the data into features and targets
y_sys = df['BP_Sys']
y_dia = df['BP_Dia']

# Split into training and testing sets for systolic
X_train_sys, X_test_sys, y_train_sys, y_test_sys = train_test_split(X_with_age_gender, y_sys, test_size=0.2, random_state=42)

# Initialize RandomForestRegressor with default hyperparameters for systolic
rf_sys = RandomForestRegressor(random_state=42)

# Perform grid search for RandomForestRegressor for systolic
rf_param_grid_sys = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15]
}

rf_grid_search_sys = GridSearchCV(estimator=rf_sys, param_grid=rf_param_grid_sys, scoring='neg_mean_absolute_error', cv=5)
rf_grid_search_sys.fit(X_train_sys, y_train_sys)
best_rf_sys = rf_grid_search_sys.best_estimator_

# Initialize GradientBoostingRegressor for systolic
gb_sys = GradientBoostingRegressor(random_state=42)

# Define the parameter grid for grid search
gb_param_grid_sys = {
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.05, 0.1, 0.2]
}

# Perform grid search for GradientBoostingRegressor for systolic
gb_grid_search_sys = GridSearchCV(estimator=gb_sys, param_grid=gb_param_grid_sys, scoring='neg_mean_absolute_error', cv=5)
gb_grid_search_sys.fit(X_train_sys, y_train_sys)
best_gb_sys = gb_grid_search_sys.best_estimator_

# Combine RandomForestRegressor and GradientBoostingRegressor predictions for systolic
ensemble_preds_sys = (best_rf_sys.predict(X_test_sys) + best_gb_sys.predict(X_test_sys)) / 2.0

# Calculate ensemble MAE for systolic prediction
ensemble_mae_sys = mean_absolute_error(y_test_sys, ensemble_preds_sys)
print("Ensemble MAE (Sys):", ensemble_mae_sys)

# Split into training and testing sets for diastolic
X_train_dia, X_test_dia, y_train_dia, y_test_dia = train_test_split(X_with_age_gender, y_dia, test_size=0.2, random_state=42)

# Initialize RandomForestRegressor with default hyperparameters for diastolic
rf_dia = RandomForestRegressor(random_state=42)

# Perform grid search for RandomForestRegressor for diastolic
rf_param_grid_dia = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15]
}

rf_grid_search_dia = GridSearchCV(estimator=rf_dia, param_grid=rf_param_grid_dia, scoring='neg_mean_absolute_error', cv=5)
rf_grid_search_dia.fit(X_train_dia, y_train_dia)
best_rf_dia = rf_grid_search_dia.best_estimator_

# Initialize GradientBoostingRegressor for diastolic
gb_dia = GradientBoostingRegressor(random_state=42)

# Define the parameter grid for grid search
gb_param_grid_dia = {
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.05, 0.1, 0.2]
}

# Perform grid search for GradientBoostingRegressor for diastolic
gb_grid_search_dia = GridSearchCV(estimator=gb_dia, param_grid=gb_param_grid_dia, scoring='neg_mean_absolute_error', cv=5)
gb_grid_search_dia.fit(X_train_dia, y_train_dia)
best_gb_dia = gb_grid_search_dia.best_estimator_

# Combine RandomForestRegressor and GradientBoostingRegressor predictions for diastolic
ensemble_preds_dia = (best_rf_dia.predict(X_test_dia) + best_gb_dia.predict(X_test_dia)) / 2.0

# Calculate ensemble MAE for diastolic prediction
ensemble_mae_dia = mean_absolute_error(y_test_dia, ensemble_preds_dia)
print("Ensemble MAE (Dia):", ensemble_mae_dia)
